<a href="https://colab.research.google.com/github/fahrulrozi11/Machine-Learning/blob/main/Boosting_or_Bagging_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.IMPORT LIB

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, classification_report
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')

2. Loading Dataset

In [9]:
def load_and_clean_data(dataset_name):
    def load_dataset(name, url, default_target):
        print(f"\n{'='*50}")
        print(f"Memproses dataset: {name}")
        print(f"{'='*50}")

        df = pd.read_csv(url)
        print(f"Jumlah data: {df.shape[0]}")
        print(f"Jumlah fitur: {df.shape[1] - 1}")
        print(f"Nilai kosong: {df.isnull().sum().sum()}")
        print("\nKolom dalam dataset:", df.columns.tolist())

        # Deteksi kolom target
        target_col = next((col for col in [default_target, default_target.lower()] if col in df.columns), None)
        if target_col is None:
            target_col = next((col for col in df.columns if df[col].nunique() <= 5), df.columns[-1])

        print(f"\nMenggunakan '{target_col}' sebagai kolom target")
        print("Distribusi Target:")
        print(df[target_col].value_counts(normalize=True))

        # Ubah target menjadi numerik jika belum
        if not pd.api.types.is_numeric_dtype(df[target_col]):
            df[target_col] = LabelEncoder().fit_transform(df[target_col])
            print("\nKolom target telah diubah menjadi nilai numerik")

        # Pisahkan fitur dan target
        X = df.drop(columns=target_col)
        y = df[target_col]

        # Kategorisasi tipe kolom
        numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

        print("\nFitur numerik:", numerical_cols)
        print("Fitur kategorikal:", categorical_cols)

        return X, y, numerical_cols, categorical_cols

    # Peta dataset ke URL dan target default
    dataset_map = {
        'heart': ("Heart Disease",
                  "https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/01.%20Machine%20Learning/01.%20Week%201/Dataset/HeartDisease.csv",
                  "target"),
        'water': ("Citarum Water",
                  "https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/01.%20Machine%20Learning/02.%20Week%202/Dataset/CitarumWater.csv",
                  "Class"),
        'income': ("Income",
                   "https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/02.%20Deep%20Learning/Dataset/income.csv",
                   "income")
    }

    if dataset_name in dataset_map:
        name, url, target = dataset_map[dataset_name]
        return load_dataset(name, url, target)
    else:
        raise ValueError("Nama dataset tidak dikenali. Gunakan 'heart', 'water', atau 'income'.")


3. Preprocessing

In [3]:
def build_preprocessing_pipeline(numerical_cols, categorical_cols):
    # Langkah pra-pemrosesan untuk kolom numerik: imputasi dengan median, lalu normalisasi
    numeric_pipeline = Pipeline([
        ('isi_kosong', SimpleImputer(strategy='median')),
        ('normalisasi', StandardScaler())
    ])

    # Langkah pra-pemrosesan untuk kolom kategorikal: imputasi dengan modus, lalu one-hot encoding
    categorical_pipeline = Pipeline([
        ('isi_kosong', SimpleImputer(strategy='most_frequent')),
        ('encoding', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Gabungkan pipeline numerik dan kategorikal ke dalam ColumnTransformer
    preprocessing_pipeline = ColumnTransformer([
        ('numerik', numeric_pipeline, numerical_cols),
        ('kategori', categorical_pipeline, categorical_cols)
    ])

    return preprocessing_pipeline


4. Training & Evaluasi Model Boosting

In [4]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, preprocessor, model_name='xgboost'):
    # Pilih model berdasarkan nama yang diberikan
    if model_name == 'xgboost':
        estimator = xgb.XGBClassifier(random_state=42)
        tuning_params = {
            'estimator__learning_rate': [0.01, 0.1],
            'estimator__max_depth': [3, 5],
            'estimator__n_estimators': [100, 200]
        }
    elif model_name == 'lightgbm':
        estimator = lgb.LGBMClassifier(random_state=42)
        tuning_params = {
            'estimator__learning_rate': [0.01, 0.1],
            'estimator__num_leaves': [31, 50],
            'estimator__n_estimators': [100, 200]
        }
    else:
        estimator = GradientBoostingClassifier(random_state=42)
        tuning_params = {
            'estimator__learning_rate': [0.01, 0.1],
            'estimator__max_depth': [3, 5],
            'estimator__n_estimators': [100, 200]
        }

    # Gabungkan preprocessing dan model ke dalam pipeline
    full_pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('estimator', estimator)
    ])

    print(f"\n🔍 Menjalankan Grid Search untuk {model_name}...")

    # Lakukan pencarian parameter terbaik
    search = GridSearchCV(
        full_pipeline,
        param_grid=tuning_params,
        cv=3,
        scoring='accuracy',
        n_jobs=-1
    )

    search.fit(X_train, y_train)
    print(f"✅ Parameter terbaik: {search.best_params_}")

    # Ambil pipeline terbaik dari hasil pencarian
    best_model = search.best_estimator_

    # Prediksi data uji
    predictions = best_model.predict(X_test)

    # Jika klasifikasi biner, coba hitung AUC
    try:
        proba = best_model.predict_proba(X_test)[:, 1] if len(np.unique(y_test)) == 2 else None
        auc = roc_auc_score(y_test, proba) if proba is not None else None
    except:
        proba = None
        auc = None
        print("⚠️ Gagal menghitung AUC. Model mungkin tidak mendukung predict_proba.")

    # Kumpulkan metrik evaluasi
    eval_metrics = {
        'accuracy': accuracy_score(y_test, predictions),
        'precision': precision_score(y_test, predictions, average='weighted'),
        'recall': recall_score(y_test, predictions, average='weighted'),
        'f1': f1_score(y_test, predictions, average='weighted'),
        'auc': auc
    }

    print("\n📊 Hasil Evaluasi:")
    for key, val in eval_metrics.items():
        if val is not None:
            print(f"{key}: {val:.4f}")

    print("\n📄 Classification Report:")
    print(classification_report(y_test, predictions))

    return best_model, eval_metrics, predictions, proba


Visualisasi data

In [8]:


import matplotlib.pyplot as plt
import seaborn as sns

def visualize_data(X, y):
    # Histogram untuk fitur numerik
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    for col in numerical_cols:
        plt.figure(figsize=(8, 6))
        sns.histplot(X[col], kde=True)
        plt.title(f'Distribusi {col}')
        plt.show()

    # Boxplot untuk fitur numerik
    for col in numerical_cols:
        plt.figure(figsize=(8, 6))
        sns.boxplot(x=y, y=X[col])
        plt.title(f'Boxplot {col} berdasarkan target')
        plt.show()

    # Countplot untuk fitur kategorikal
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    for col in categorical_cols:
        plt.figure(figsize=(8, 6))
        sns.countplot(x=X[col], hue=y)
        plt.title(f'Distribusi {col} berdasarkan target')
        plt.xticks(rotation=45, ha='right')  # Rotasi label sumbu x untuk keterbacaan
        plt.show()

    # Heatmap korelasi fitur numerik
    plt.figure(figsize=(10, 8))
    sns.heatmap(X[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Korelasi antar fitur numerik')
    plt.show()


:# Penjelasan Matematika: Boosting dan Metrik Evaluasi Klasifikasi

## A. Boosting untuk Klasifikasi

Boosting merupakan metode ensemble yang mengombinasikan beberapa model lemah (base learners) secara iteratif untuk membangun model yang lebih kuat. Secara umum, model boosting dapat dituliskan sebagai:
$$
F(x) = F_0(x) + \sum_{m=1}^{M} \gamma_m h_m(x)
$$
di mana:
- $F_0(x)$ adalah prediksi awal (misalnya probabilitas dasar atau log odds),
- $h_m(x)$ adalah base learner pada iterasi ke-$m$ (misalnya decision tree),
- $\gamma_m$ adalah bobot learning rate yang mengatur kontribusi masing-masing base learner.

Pada setiap iterasi, boosting mengupdate model dengan memfokuskan pada error yang masih tersisa dari model sebelumnya.

## B. Metrik Evaluasi Klasifikasi

Evaluasi performa model klasifikasi dilakukan dengan beberapa metrik sebagai berikut:

1. **Accuracy**  
   $$
   \text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
   $$
   Accuracy mengukur proporsi prediksi yang benar.

2. **Precision**  
   $$
   \text{Precision} = \frac{TP}{TP+FP}
   $$
   Precision menunjukkan akurasi prediksi positif.

3. **Recall**  
   $$
   \text{Recall} = \frac{TP}{TP+FN}
   $$
   Recall mengukur kemampuan model menangkap kasus positif.

4. **F1 Score**  
   F1 score adalah rata-rata harmonis dari precision dan recall:
   $$
   F1 = 2 \cdot \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   $$
   Untuk analisis tambahan, kita juga dapat menghitung **F1 Squared** sebagai:
   $$
   F1^2 = (F1)^2
   $$

5. **ROC Curve dan AUC**  
   ROC (Receiver Operating Characteristic) menggambarkan trade-off antara True Positive Rate (Recall) dan False Positive Rate:
   $$
   \text{TPR} = \frac{TP}{TP+FN}, \quad \text{FPR} = \frac{FP}{FP+TN}
   $$
   AUC (Area Under Curve) mengukur luas di bawah ROC curve sehingga semakin mendekati 1, semakin baik performa model.

Metrik terbaik biasanya ditentukan berdasarkan konteks aplikasinya. Sebagai contoh, pada deteksi penyakit (misalnya HeartDisease) Recall seringkali menjadi metrik penting, sedangkan pada aplikasi lain (misalnya income prediction) AUC bisa memberikan gambaran yang lebih komprehensif terhadap performa model.
